### 1) Import Required Libraries

In [16]:
# Import required libraries 

import numpy as np
import pandas as pd
from tqdm import tqdm 
import warnings
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import concurrent.futures
warnings.filterwarnings('ignore')

from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout,\
                                    TimeDistributed, Conv1D, MaxPooling1D,\
                                    Flatten, Bidirectional, Input, Flatten,\
                                    Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost as xgb


### 2) Load Data

In [2]:
# load data 

train_dataset = pd.read_csv('inputs/train.csv',index_col='cfips')
revealed_data = pd.read_csv('inputs/revealed_test.csv',index_col='cfips')

cfips = train_dataset.index.unique()

new_df = pd.DataFrame(columns=train_dataset.columns).astype(train_dataset.dtypes)

for cfip in cfips: 
    new_df = pd.concat([new_df,train_dataset.loc[cfip], revealed_data.loc[cfip]])

train_dataset = new_df

### 3) Preprocessing 

In [3]:
train_dataset.index = train_dataset.index.astype("string")
train_dataset['first_day_of_month'] = pd.to_datetime(train_dataset.first_day_of_month, format='%Y-%m-%d')

In [4]:
# Number of rows for each county 
cfips = train_dataset.index.unique()
cfip = cfips[0]
total_counties = len(cfips)
len(train_dataset)/len(train_dataset.loc[cfip]) == total_counties, len(train_dataset), len(train_dataset.loc[cfip])

(True, 128535, 41)

### 4) Outliers detection in population and MBD

In [ ]:
train_dataset["population"] = train_dataset["microbusiness_density"]
train_dataset["population"] = 100*train_dataset["active"]/train_dataset["microbusiness_density"]
train_dataset["population"] = train_dataset["population"].ffill()
train_dataset["population"] = train_dataset["population"].apply(lambda x : int(x))

In [ ]:
train_dataset.describe()

In [ ]:
# Checking counties with abnormal variation in population

def is_pop_outlier(cfip):
    flag = False
    data = train_dataset.loc[cfip].population.values

    # calculate the Z-score for each data point
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    
    # identify any data points with a Z-score greater than 3 (considered outliers)
    outliers = data[z_scores > 3]
    
    if len(outliers)>0: 
        print(f"cfip -> {cfip} : Found {len(outliers)} outliers.")
        flag = True
    return flag
    
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(is_pop_outlier,cfip) for cfip in cfips]
    results = [f.result() for f in concurrent.futures.as_completed(futures)]
    
print("number of counties with outliers:" , sum(results))

In [ ]:
train_dataset.loc["28055"].tail(5)

In [ ]:
# Assuming counties with low population will have large variation in MBD on even small change in `active` 
# business , we will only consider counties having atleast population = POPULATION_THRESHOLD

# Filtering counties with low population
POPULATION_THRESHOLD = 5000        

def above_threshold(cfip):
    return train_dataset.loc[cfip].population.mean()>POPULATION_THRESHOLD

low_population_counties = [cfip for cfip in cfips if not above_threshold(cfip)]
high_population_counties = [cfip for cfip in cfips if cfip not in low_population_counties]
print("Number of counties with low population : ", len(low_population_counties))

In [ ]:
# Check for null valus 
train_dataset.isna().sum()

In [ ]:
# Checking counties with abnormal variation in MBD and fix them by substituting them with mbd values with 
# previous month

def is_mbd_outlier(cfip,fix=False):
    data = train_dataset.loc[cfip].microbusiness_density.values
    # calculate the Z-score for each data point
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    
    # identify any data points with a Z-score greater than 3 (considered outliers)
    outliers = data[z_scores > 3]
    
    if len(outliers)>0: 
        # print(f"cfip -> {cfip} : Found {len(outliers)} outliers.")
        
        # fix the outliers by replacing them with the their previous month values
        if fix: 
            flags = z_scores>3
            for i in range(1,len(data)): 
                if flags[i] : 
                    data[i] = data[i-1]
        
            # data[z_scores>3] = np.median(data)  # alternate way if above doesn't work 
            
            train_dataset.loc[cfip,"microbusiness_density"] = data
        return 1
    return 0
    
results = []
for cfip in cfips:   
    results.append(is_mbd_outlier(cfip,fix=True))
print("number of counties with outliers in mbd (before fixing):" , sum(results))

### 5) Utils Functions and Variables

In [5]:
input_size = 8
output_size = 2

In [6]:

def split_data(data, input_size, output_size):
    """
    Splits the time-series data into train, validation, and test sets, with a window size of input_size as the input
    and output_size as the output.
    """
    # MinMax Scaler
    # scaler = MinMaxScaler()
    scaler = StandardScaler()
    data = scaler.fit_transform(data.values.reshape(-1,1))
    # Calculate the total number of windows
    num_windows = len(data) - input_size - output_size + 1
    
    # Split the data into input windows and output windows
    inputs = np.zeros((num_windows, input_size,1))
    outputs = np.zeros((num_windows, output_size,1))
    
    for i in range(num_windows):
        inputs[i] = np.array(data[i:i+input_size]).reshape(-1,1)
        outputs[i] = np.array(data[i+input_size:i+input_size+output_size]).reshape(-1,1)
        
    # Split the data into train, validation, and test sets
    num_train = int(0.8 * num_windows)
    num_val = num_windows - num_train
    num_test = num_windows - num_train - num_val  # test size = 0 
    
    train_inputs = inputs[:num_train]
    train_outputs = outputs[:num_train]
    
    val_inputs = inputs[num_train:num_train+num_val]
    val_outputs = outputs[num_train:num_train+num_val]
    
    test_inputs = inputs[num_train+num_val:]
    test_outputs = outputs[num_train+num_val:]
    
    prediction_inputs = np.array(data[-input_size:]).reshape(1,-1,1)
    return train_inputs, train_outputs, val_inputs, val_outputs, \
           test_inputs, test_outputs, prediction_inputs, scaler


In [7]:
# Evaluation Function : smape ( symmetric mean absolute percentage error )
def smape(A,F): 
    """
    calculate SMAPE ( Symmetric Mean Absolute Percentage Error) value
    
    Inputs: 
        A - actual value(s)
        F - forecast value(s) 
    """  
    if type(A)==np.ndarray and type(F)==np.ndarray: 
        if(len(A) != len(F)): 
            print("Shapes didn't matched")
            return 
    a = np.abs(A-F)
    b = (np.abs(A) + np.abs(F))/2
    if type(A)==np.ndarray and type(F)==np.ndarray: 
        result = np.divide(a,b, out=np.zeros_like(a,dtype=type(a)), where=b!=0)
        result = np.sum(result,axis=0)/len(result)
    else: 
        result = a/b if b!=0 else np.zeros(1)
    return result 

### 6) Building Model 

In [8]:
# Define callbacks 

# Model checkpoint - To save the model at certain frequencies 
checkpoint_filepath = "./saved_models/checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath=checkpoint_filepath,
                                        save_weights_only=False,
                                        monitor="val_loss",
                                        mode="min",
                                        save_best_only=True)


# Early stopping - stopping evaluation if monitored evaluation metric is no longer improving
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
                                            monitor="val_loss",
                                            min_delta=0.005,
                                            patience=10,
                                            mode="min")

# Reduce learning rate - decreasing the learning rate when the monitored metric has stopped improving
rlrop_callback = tf.keras.callbacks.ReduceLROnPlateau(
                                            monitor="val_loss", 
                                            factor=0.2, 
                                            mode="min", 
                                            patience=3, 
                                            min_lr=0.001)

In [9]:
# Creating a model that takes this time series and output future predictions

# Model Artchitecture - An encoder-decoder model 
# (source - https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668)

def create_model1():

    model = Sequential()
    model.add(GRU(70,activation="relu",input_shape=(input_size,1)))
    model.add(RepeatVector(output_size))
    model.add(GRU(70,activation="relu",return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer="adam",loss="mse")
    # plot_model(model=model,show_shapes=True)
    return model 

def create_model2():

    # Define the model architecture
    model = Sequential()

    # Add the CNN layers 
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_size, 1)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Add the LSTM layers
    model.add(LSTM(units=128, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=64, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=32, activation='relu'))

    # Add the output layer
    model.add(Dense(units=output_size))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    return model 



In [10]:
def inverse_transform(scaler, y_test, yhat):
 y_test_reshaped = y_test.reshape(-1, y_test.shape[1])
 yhat_reshaped = yhat.reshape(-1, yhat.shape[1])
 yhat_inverse = scaler.inverse_transform(yhat_reshaped)
 y_test_inverse = scaler.inverse_transform(y_test_reshaped)
 return yhat_inverse, y_test_inverse

In [11]:
with open('results.csv','w') as fp: 
    fp.write('cfips,val_smape,pred_1,pred_2\n')

### 7) Train Model and Make Predictions

In [20]:
epochs = 100
batch_size = 32

def run_model(cfip,model=None):
    if not model : 
        model = create_model2()
    
    train_inputs, train_outputs, val_inputs, val_outputs, test_inputs, test_outputs, prediction_inputs,scaler = \
        split_data(train_dataset.loc[cfip].microbusiness_density, input_size, output_size)
        
    model.fit(train_inputs,train_outputs,
            epochs=epochs,batch_size=batch_size, 
            validation_data=(val_inputs,val_outputs),
            callbacks=[early_stopping_callback, rlrop_callback],
            verbose=0)    
    model.verbose = False
    predictions = model.predict(val_inputs,verbose=0)
    predictions, val_outputs = inverse_transform(scaler,predictions,val_outputs)
    val_smape = np.mean(smape(predictions,val_outputs))
    # future_predictions = scaler.inverse_transform(model.predict(prediction_inputs,verbose=0))
    future_predictions = list(scaler.inverse_transform(model.predict(prediction_inputs,verbose=0).reshape(-1,output_size)).reshape(-1))
    
    with open('results.csv','a') as fp : 
        fp.write('{},{}\n'.format(cfip,val_smape))
    return cfip, val_smape, future_predictions
    
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     subset = cfips[:20]
#     results = list(tqdm(executor.map(run_model,subset),total=len(subset)))

In [14]:
# results

### 8) Submit predictions

In [ ]:
# Save the output to submission_file.csv

# def submit():
df = pd.read_csv('results.csv')
submission = pd.read_csv('inputs/sample_submission.csv')
submission['cfips'] = submission['row_id'].apply(lambda x : x.split('_')[0])
df.cfips = df.cfips.astype('string')
submission.cfips = submission.cfips.astype('string')
submission = submission.set_index('cfips')
df.set_index('cfips',inplace=True)

In [ ]:
cfips = submission.index.unique()
for cfip in cfips: 
    submission.loc[cfip,"microbusiness_density"] = df.loc[cfip,"pred_1"]

In [ ]:
submission.to_csv("submission_2_march.csv",index=False)

In [ ]:
submission

In [ ]:
submission.to_csv('submission_2_march.csv',index=False)

In [ ]:
submission["cfips"] = submission["row_id"].apply(lambda x : x.split("_")[0])

In [ ]:
submission.set_index("cfips",inplace=True)

In [ ]:
submission.loc["1013"]

### 9) Ensemble Model - GRU + XGBoost

In [24]:
# define the CNN-GRU model
def cnn_lstm_model(n_input, n_output, n_filters, filter_size, n_lstm_units, n_lstm_layers, dropout_rate):
    inputs = Input(shape=(n_input,1))

    # CNN Layers
    conv1 = Conv1D(filters=n_filters, kernel_size=filter_size, activation='relu')(inputs)
    lstm_in = Dropout(rate=dropout_rate)(conv1)

    # GRU Layers
    lstm1 = GRU(units=n_lstm_units, return_sequences=True)(lstm_in)
    for i in range(n_lstm_layers-1):
        lstm1 = GRU(units=n_lstm_units, return_sequences=True)(lstm1)
        lstm1 = Dropout(rate=dropout_rate)(lstm1)
    lstm_out = GRU(units=n_lstm_units)(lstm1)

    # Output Layer
    outputs = Dense(n_output)(lstm_out)

    # create and compile the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    
    return model

# define the XGBoost model
def xgb_model_create(n_estimators, max_depth):
    model = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
    return model

# define the combined model
def cnn_lstm_xgb_model(n_input, n_output, n_filters, filter_size, n_lstm_units, n_lstm_layers, dropout_rate, n_estimators, max_depth):
    # define the CNN-LSTM model
    cnn_lstm = cnn_lstm_model(n_input, n_output, n_filters, filter_size, n_lstm_units, n_lstm_layers, dropout_rate)

    # define the XGBoost model
    xgb_model = xgb_model_create(n_estimators, max_depth)

    # define the inputs
    inputs = Input(shape=(n_input,1))

    # pass the inputs through the CNN-LSTM model
    cnn_lstm_out = cnn_lstm(inputs)

    # pass the CNN-LSTM output through the XGBoost model
    xgb_out = xgb_model(cnn_lstm_out)

    # create and compile the final model
    model = Model(inputs=inputs, outputs=xgb_out)
    model.compile(optimizer='adam', loss='mse')

    return model


In [30]:
model = cnn_lstm_xgb_model(input_size,output_size,64,2,50,5,0.2,30,5)
# run_model("1001",model)

TypeError: 'XGBRegressor' object is not callable